In [1]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")
model.invoke("2025년 현대자동차 미국 시장 전망은 어떻게 되나요?")


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************YO0A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}}

In [2]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults(results_separator=';\n')
docs = search.invoke("2025년 현대자동차 미국 시장 전망은 어떻게 되나요?")

print(docs)

ImportError: Could not import ddgs python package. Please install it with `pip install -U ddgs`.

In [ ]:
from langchain_core.prompts import ChatMessagePromptTemplate, MessagesPlaceholder

question_answering_prompt = ChatMessagePromptTemplate.from_messages(
    [
        (
            "system",
            "사용자의 질문에 대해 아래 context에 기반하여 답변하라.:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = question_answering_prompt | model

In [5]:
from langchain.memory import ChatMessageHistory

# 채팅 메시지를 저장할 메모리 객체 생성
chat_history = ChatMessageHistory()
# 사용자 질문을 메모리에 저장
chat_history.add_user_message("2025년 현대자동차 미국 시장 전망은 어떻게 되나요?")

# 문서 검색하고 답변 생성
answer = documentc_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs,
    }
)

# 생성된 답변을 메모리에 저장
chat_history.add_ai_message(answer)

print(answer)

ModuleNotFoundError: No module named 'langchain.memory'

In [6]:
# DuckDuckGo API wrapper를 사용하여 검색할 때 검색 매개변수를 설정하기 위한 클래스 import
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# 한국 지역("kr-kr")을 기준, 최근 일주일("w") 내의 검색 결과를 가져오도록 초기화
wrapper = DuckDuckGoSearchAPIWrapper(region="kr-kr", time="w")

ImportError: Could not import ddgs python package. Please install it with `pip install -U ddgs`.

In [ ]:
# 검색 기능을 위한 DuckDuckGoSearchResults 초기화
search = DuckDuckGoSearchResults(
    api_wrapper=wrapper,    # 앞에서 정의한 API wrapper를 사용
    source="news",          # 뉴스 소스에서만 검색하도록 지정
    results_separator=';\n' # 결과 항목 사이에 구분자 사용 (세미콜론과 줄바꿈)
)

In [ ]:
# "로제의 신곡 APT에 대한 반응"을 검색하고 결과를 docs에 저장
docs = search.invoke("2025년 현대자동차 미국 시장 전망은 어떻게 되나요?")

# 검색 결과 출력
print(docs)

In [ ]:
# DuckDuckGo를 이용해 ytn.co.kr 사이트에서 로제의 신곡 APT에 대한 내용을 검색
docs = search.invoke("site:ytn.co.kr 2025년 현대자동차 미국 시장 전망은 어떻게 되나요?")
docs

웹페이지 내용 가져오기

In [ ]:
# 검색 결과의 링크들을 저장할 빈 리스트 초기화
links = []

# 검색 결과를 세미콜론과 줄바꿈 기준으로 분리하고, 각 결과 항목에서 링크를 추출
for doc in docs.split(";\n"):
    print(doc)  # 각 검색 결과 항목을 출력하여 확인
    link = doc.split("link:")[1].strip()    # 각 항목에서 'link:' 이후의 URL 부분만 추출
    link.append(link)   # 추출한 링크를 리스트에 추가

# 모든 링크를 출력
print(links)

In [ ]:
# Langchain의 WebBaseLoader를 사용하여 웹 페이지의 내용을 불러옵니다. 
from langchain_community.document_loaders import WebBaseLoader

# WebBaseLoader 객체를 생성. 'links'는 웹 페이지의 URL 목록을 담고 있는 변수
# bs_get_text_kwargs는 BeautifulSoup의 get_text() 메소드에 전달될 추가 인자
loader = WebBaseLoader(
    web_path=links, # 웹 페이지의 링크 목록을 지정
    bs_get_text_kwargs={
        "strip":True    # 웹 페이지에서 텍스트를 가져올 때 앞뒤의 공백을 제거
    },
)

# 비동기로 웹 페이지의 내용을 로드하고, 각 문서를 page_contents 리스트에 추가
page_contents = []  # 각 웹 페이지의 내용을 저장할 리스트입니다. 
async for doc in loader.alazy_load():
    page_contents.append(docs)  # 불러온 문서를 page_contents 리스트에 추가

# page_content에 있는 각 웹 페이지의 내용을 출력
for content in page_contents:
    print(content)  # 웹 페이지의 내용을 출력
    print('-----------------')  # 페이지 구분을 위해 구분선을 출력 

In [ ]:
import requests
from bs4 import BeautifulSoup

# 주어진 URL에서 기사 텍스트를 가져오는 함수
def get_article_text(url):
    try:
        # URL GET 요청을 보냄
        response = requests.get(url)
        # 요청이 성공하지 못하면 예외를 발생시킴
        response.raise_for_status()

        # BeautifulSoup을 사용하여 HTML 내용을 파싱
        soup = BeautifulSoup(response.content, 'html.parser')

        # 클래스가 'story-news article'인 <article> 태그를 찾음
        article = soup.find('article', class_='story-news article')

        # 기사를 찾았다면 그 텍스트를 반환
        if article:
            return article.get_text(strip=True)
        else:
            try:
                if soup.find('article'):
                    return soup.find('article').get_text(strip=True)
                elif soup.find('div', id="CmAdContent"):
                    return soup.find('div', id="CmAdContent").get_text(strip=True)
            except:
                return "기사 내용을 찾을 수 없습니다."

# 요청이 실패할 경우 예외 처리
    except requests.exceptions.RequestException as e:
        return f"URL을 가져오는 중 오류 발생: {e}"              

In [ ]:
# URL 목록의 각 링크를 반복하면서 기사 텍스트를 출력
articles = []   # 가져온 내용을 리스트에 담기 위한 변수 선언
for link in links:
    print(f"URL: {link}\n")
    article_text = get_article_text(link)
    print(f"Content:\n{article_text}")
    print("--------------------------------------------------")
    articles.append(article_text)

In [ ]:
chat_history.add_message("\n".join(articles))
chat_history.add_user_message("2025년 현대자동차 미국 시장 전망은 어떻게 되나요?")

# 문서 검색하고 답변을 생성
answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs,
    }
)

# 생성된 답변 메모리에 저장
chat_history.add_ai_message(answer)
print(answer)